# Project Overview

Our project's objective is to achieve precise localization for a car driving within a simulated environment. The task involves ensuring that the car can travel a minimum distance of 170 meters from its starting position while minimizing the distance pose error. Practically, this error should not exceed 1.2 meters.

To accomplish this, we will utilize point clouds extracted from a simulated car equipped with a LIDAR sensor, which provides regular LIDAR scans. Additionally, we have an existing point cloud map, which has been extracted from the CARLA simulator. Localization will be achieved by performing point registration matching between the real-time LIDAR scans and the pre-existing point cloud map. For this we implemented the ICP and NTD localization algorithm respectively.

# Set Up
TODO: Write explaination about the steps to run the code

# ICP Localization

The Iterative Closest Point (ICP) algorithm is used to align two sets of points, typically referred to as the source points \( P \) and the target points \( Q \). In the context of a self-driving car, the target points \( Q \) are provided by a map, while the source points \( P \) come from the car's sensors. The goal is to transform the car's sensor data so it aligns with the map coordinates, specifically aligning the x and y coordinates through translation and rotation. This is achieved by minimizing the sum of squared errors between the points. This is achieved iteratively. 

One iteration includes the following steps:

1. Calculate the centroids (mean points) of both \( P \) and \( Q \).

2. Create a new set \( X \) by subtracting each point in \( P \) by the centroid of \( P \), and create a new set \( Y \) by subtracting each point in \( Q \) by the centroid of \( Q \).

3. Compute the cross covariance matrix \( S \) by multiplying \( X \) with the transpose of \( Y \).

4. Perform SVD on the matrix \( S \) to obtain matrices \( U \), \( \Sigma \), and \( V^T \).

5. 
    - The rotation matrix \( R \) is calculated as \( R = VU^T \).
    - The translation vector \( t \) is calculated as \( t = \mu_Q - R\mu_P \), where \( \mu_Q \) and \( \mu_P \) are the centroids of \( Q \) and \( P \), respectively.

6. We then iterativeley repeat the above steps until the sum of squared errors is minimized to a satisfactory level.

By iteratively updating the transformation, the ICP algorithm aligns the source points \( P \) to the target points \( Q \), effectively mapping the car's sensor data to the map coordinates.

## Module configuration

TODO: Explain the module configuration and selected parameters

## Dataset performance metrics

On the provided dataset, our model demonstrated the following performance metrics:

- **Mean time per frame**: 0.12 seconds
- **Maximal time per frame**: 0.59 seconds
- **Mean lateral error**: 0.07 meters
- **Maximal lateral error**: 0.07 meters


## Code

In [2]:
# TODO: Update paths, remove final part about visualization?

import open3d as o3d
import numpy as np
import copy
import time
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import pandas as pd
import os

#
# Paths
map_path = "localization/dataset/map.pcd"
frames_path = "localization/dataset/frames"
ground_truth_path = "localization/dataset/ground_truth.csv"

# point cloud
map_cloud = o3d.io.read_point_cloud(map_path)

# ground truth
ground_truth = pd.read_csv(ground_truth_path)

extracted_cars = []

#frame
for i in range(len(ground_truth)):
    frame_file = os.path.join(frames_path, f"frame_{i}.pcd")

    if not os.path.exists(frame_file):
        print(f"Frame file {frame_file} not found.")
        continue

    frame_cloud = o3d.io.read_point_cloud(frame_file)
    frame_cloud_down = frame_cloud.voxel_down_sample(voxel_size=0.05)

    # ground truth
    x, y, z = ground_truth.iloc[i][1:4]
    roll, pitch, yaw = np.deg2rad(ground_truth.iloc[i][4:7])

    # transformation matrix
    r = R.from_euler('xyz', [roll, pitch, yaw])
    rotation_matrix = r.as_matrix()
    translation_vector = np.array([x, y, z])
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = translation_vector

    # transformation
    frame_cloud_down.transform(transformation_matrix)
    extracted_cars.append(frame_cloud_down)

# ICP
threshold = 0.1
initial_transformation = np.eye(4)
maximum_iteration = 100

time_rec = []
lateral_err = []


for i, car_cloud in enumerate(extracted_cars):
    start_time = time.time()

    reg_p2l = o3d.pipelines.registration.registration_icp(
        car_cloud, map_cloud, threshold, initial_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=maximum_iteration)
    )

    end_time = time.time()
    time_rec.append(end_time - start_time)

    # registration error
    lateral_error = o3d.pipelines.registration.evaluate_registration(
        car_cloud, map_cloud, threshold, reg_p2l.transformation
    ).inlier_rmse
    lateral_err.append(lateral_error)

    if lateral_error > 1.2:
        print(f"Lateral error ({lateral_error:.2f} m) is greater than the maximum allowed (1.2 m).")
        break

    # initial transformation
    initial_transformation = reg_p2l.transformation

    print(f"Processed {i + 1}/{len(extracted_cars)} frames.")

# results
print(f"----->>>>>>>>>>  Mean time per frame: {np.mean(time_rec):.2f} s")
print(f"----->>>>>>>>>>  Maximal time per frame: {np.max(time_rec):.2f} s")
print(f"----->>>>>>>>>>  Mean lateral error: {np.mean(lateral_err):.2f} m")
print(f"----->>>>>>>>>>  Maximal lateral error: {np.max(lateral_err):.2f} m")


# aligned car point clouds
merged_cloud = map_cloud
for car_cloud in extracted_cars:
    merged_cloud += car_cloud

o3d.io.write_point_cloud("aligned_car.pcd", merged_cloud)
# # ######################################################### End of solution #########################################################

# After saving the aligned point cloud, you can visualize it using the following code:
# pcd = o3d.io.read_point_cloud("/home/ari/Workplace/JKU/SEM_2/Autonomous_sys/Project3/Localization_Project/localization/Project/ICP/aligned_car.pcd")
# o3d.visualization.draw_geometries([pcd])



Processed 1/1014 frames.
Processed 2/1014 frames.
Processed 3/1014 frames.
Processed 4/1014 frames.
Processed 5/1014 frames.
Processed 6/1014 frames.
Processed 7/1014 frames.
Processed 8/1014 frames.
Processed 9/1014 frames.
Processed 10/1014 frames.
Processed 11/1014 frames.
Processed 12/1014 frames.
Processed 13/1014 frames.
Processed 14/1014 frames.
Processed 15/1014 frames.
Processed 16/1014 frames.
Processed 17/1014 frames.
Processed 18/1014 frames.
Processed 19/1014 frames.
Processed 20/1014 frames.
Processed 21/1014 frames.
Processed 22/1014 frames.
Processed 23/1014 frames.
Processed 24/1014 frames.
Processed 25/1014 frames.
Processed 26/1014 frames.
Processed 27/1014 frames.
Processed 28/1014 frames.
Processed 29/1014 frames.
Processed 30/1014 frames.
Processed 31/1014 frames.
Processed 32/1014 frames.
Processed 33/1014 frames.
Processed 34/1014 frames.
Processed 35/1014 frames.
Processed 36/1014 frames.
Processed 37/1014 frames.
Processed 38/1014 frames.
Processed 39/1014 fra

True

# NTD Localization

## Algorithm explaination

Unlike the Iterative Closest Point (ICP) algorithm, which minimizes point-to-point distances, NDT aligns a set of source points to a target point cloud by modeling the target as a set of normal distributions via Multivariate Normal Distribution. Since it is possible that we obtain probability density functions that dont represent our point cloud, we divide the space of the poinclouds into a grid of cells (also called voxels) and calculate the Multivariate Normal Distribution for each cell. 

 #### TODO: Explain the steps in NTD


## Module configuration
TODO: Explain the module configuration and selected parameters 

## Dataset performance metrics
TODO: Elaborate on the performance metrics attained

## Code

In [ ]:
# TODO: Copy the runable code for the NTD localization into this section


import open3d as o3d
import pandas as pd
import numpy as np
import os

class NDTRegistration:
    def __init__(self, resolution):
        self.resolution = resolution  # Voxel grid size

    def voxel_index(self, point):
        return np.floor(point / self.resolution).astype(np.int32)

    def create_voxel_grid(self, point_cloud):
        indices = np.apply_along_axis(self.voxel_index, 1, np.asarray(point_cloud.points))
        voxel_dict = {}
        for index, p in zip(indices, np.asarray(point_cloud.points)):
            index_tuple = tuple(index)
            if index_tuple not in voxel_dict:
                voxel_dict[index_tuple] = []
            voxel_dict[index_tuple].append(p)
        return voxel_dict

    def compute_voxel_stats(self, voxel_grid):
        stats = {}
        for voxel, points in voxel_grid.items():
            if len(points) > 0:
                points = np.array(points)
                mean = np.mean(points, axis=0)
                covariance = np.cov(points, rowvar=False)
                stats[voxel] = (mean, covariance)
        return stats

    def register(self, source, target):
        source_voxel = self.create_voxel_grid(source)
        target_voxel = self.create_voxel_grid(target)

        source_stats = self.compute_voxel_stats(source_voxel)
        target_stats = self.compute_voxel_stats(target_voxel)
        transformation = np.eye(4)
        return transformation

def load_ground_truth(file_path):
    return pd.read_csv(file_path, delimiter=',')

def load_pcd(frame_number, dataset_folder):
    file_path = os.path.join(dataset_folder, f"frames/frame_{frame_number}.pcd")
    if os.path.exists(file_path):
        return o3d.io.read_point_cloud(file_path)
    else:
        return None

def transform_point_cloud(pcd, x, y, z, roll, pitch, yaw):
    R = np.array([
        [np.cos(yaw)*np.cos(pitch), np.cos(yaw)*np.sin(pitch)*np.sin(roll) - np.sin(yaw)*np.cos(roll), np.cos(yaw)*np.sin(pitch)*np.cos(roll) + np.sin(yaw)*np.sin(roll)],
        [np.sin(yaw)*np.cos(pitch), np.sin(yaw)*np.sin(pitch)*np.sin(roll) + np.cos(yaw)*np.cos(roll), np.sin(yaw)*np.sin(pitch)*np.cos(roll) - np.cos(yaw)*np.sin(roll)],
        [-np.sin(pitch), np.cos(pitch)*np.sin(roll), np.cos(pitch)*np.cos(roll)]
    ])
    T = np.eye(4)
    T[:3, :3] = R
    T[0, 3] = x
    T[1, 3] = y
    T[2, 3] = z
    pcd.transform(T)
    return pcd

def main():
    dataset_folder = 'localization/dataset'
    ground_truth_file = 'localization/dataset/ground_truth.csv'
    reference_pcd_path = 'localization/dataset/reference_map.pcd'  # Path to the reference point cloud

    ground_truth = load_ground_truth(ground_truth_file)
    ground_truth.columns = [col.strip().lower() for col in ground_truth.columns]
    reference_pcd = o3d.io.read_point_cloud(reference_pcd_path)  # Load reference point cloud once

    ndt = NDTRegistration(resolution=1.0)  # Initialize NDT registration

    for index, row in ground_truth.iterrows():
        frame_number = int(row['frame'])
        source_pcd = load_pcd(frame_number, dataset_folder)
        if source_pcd is not None:
            transformed_pcd = transform_point_cloud(source_pcd, row['x'], row['y'], row['z'], row['roll'], row['pitch'], row['yaw'])
            transformation_ndt = ndt.register(transformed_pcd, reference_pcd)
            transformed_pcd.transform(transformation_ndt)  # Apply NDT transformation
            o3d.visualization.draw_geometries([transformed_pcd, reference_pcd])

if __name__ == "__main__":
    main()


# Analysis


## Comparison of both algorithms 

### Error through frames

# TODO: compare the error through frames 
ICP: 
Mean lateral error: 0.07 m


### Maximum error

# TODO: compare the maximum error of 

ICP: 
Maximal lateral error: 0.07 m


### Computation time

# TODO: Compare the computation time 

ICP: 
Mean time per frame: 0.12 s
Maximal time per frame: 0.59 s

### Complexity

# TODO: Compare the complexity of the algorithms